Trabalho trilha 08 DeepLearning

In [ ]:
import pandas as pd
from IPython.display import display
import tensorflow as tf
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving ion.csv to ion.csv


In [ ]:
ion = pd.read_csv('ion.csv', index_col=0)
display(ion.head())

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,V29,V30,V31,V32,V33,V34,Class
1,1,0,0.99539,-0.05889,0.85243,0.02306,0.83398,-0.37708,1.00000,0.03760,0.85243,-0.17755,0.59755,-0.44945,0.60536,-0.38223,0.84356,-0.38542,0.58212,-0.32192,0.56971,-0.29674,0.36946,-0.47357,0.56811,-0.51171,0.41078,-0.46168,0.21266,-0.34090,0.42267,-0.54487,0.18641,-0.45300,good
2,1,0,1.00000,-0.18829,0.93035,-0.36156,-0.10868,-0.93597,1.00000,-0.04549,0.50874,-0.67743,0.34432,-0.69707,-0.51685,-0.97515,0.05499,-0.62237,0.33109,-1.00000,-0.13151,-0.45300,-0.18056,-0.35734,-0.20332,-0.26569,-0.20468,-0.18401,-0.19040,-0.11593,-0.16626,-0.06288,-0.13738,-0.02447,bad
3,1,0,1.00000,-0.03365,1.00000,0.00485,1.00000,-0.12062,0.88965,0.01198,0.73082,0.05346,0.85443,0.00827,0.54591,0.00299,0.83775,-0.13644,0.75535,-0.08540,0.70887,-0.27502,0.43385,-0.12062,0.57528,-0.40220,0.58984,-0.22145,0.43100,-0.17365,0.60436,-0.24180,0.56045,-0.38238,good
4,1,0,1.00000,-0.45161,1.00000,1.00000,0.71216,-1.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,-1.00000,0.14516,0.54094,-0.39330,-1.00000,-0.54467,-0.69975,1.00000,0.00000,0.00000,1.00000,0.90695,0.51613,1.00000,1.00000,-0.20099,0.25682,1.00000,-0.32382,1.00000,bad
5,1,0,1.00000,-0.02401,0.94140,0.06531,0.92106,-0.23255,0.77152,-0.16399,0.52798,-0.20275,0.56409,-0.00712,0.34395,-0.27457,0.52940,-0.21780,0.45107,-0.17813,0.05982,-0.35575,0.02309,-0.52879,0.03286,-0.65158,0.13290,-0.53206,0.02431,-0.62197,-0.05707,-0.59573,-0.04608,-0.65697,good


Analise exploratoria

In [ ]:
ion.describe()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,V29,V30,V31,V32,V33,V34
count,351.000000,351.0,351.000000,351.000000,351.000000,351.000000,351.000000,351.000000,351.000000,351.000000,351.000000,351.000000,351.000000,351.000000,351.000000,351.000000,351.000000,351.000000,351.000000,351.000000,351.000000,351.000000,351.000000,351.000000,351.000000,351.000000,351.000000,351.000000,351.000000,351.000000,351.000000,351.000000,351.000000,351.000000
mean,0.891738,0.0,0.641342,0.044372,0.601068,0.115889,0.550095,0.119360,0.511848,0.181345,0.476183,0.155040,0.400801,0.093414,0.344159,0.071132,0.381949,-0.003617,0.359390,-0.024025,0.336695,0.008296,0.362475,-0.057406,0.396135,-0.071187,0.541641,-0.069538,0.378445,-0.027907,0.352514,-0.003794,0.349364,0.014480
std,0.311155,0.0,0.497708,0.441435,0.519862,0.460810,0.492654,0.520750,0.507066,0.483851,0.563496,0.494817,0.622186,0.494873,0.652828,0.458371,0.618020,0.496762,0.626267,0.519076,0.609828,0.518166,0.603767,0.527456,0.578451,0.508495,0.516205,0.550025,0.575886,0.507974,0.571483,0.513574,0.522663,0.468337
min,0.000000,0.0,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000
25%,1.000000,0.0,0.472135,-0.064735,0.412660,-0.024795,0.211310,-0.054840,0.087110,-0.048075,0.021120,-0.065265,0.000000,-0.073725,0.000000,-0.081705,0.000000,-0.225690,0.000000,-0.234670,0.000000,-0.243870,0.000000,-0.366885,0.000000,-0.332390,0.286435,-0.443165,0.000000,-0.236885,0.000000,-0.242595,0.000000,-0.165350
50%,1.000000,0.0,0.871110,0.016310,0.809200,0.022800,0.728730,0.014710,0.684210,0.018290,0.667980,0.028250,0.644070,0.030270,0.601940,0.000000,0.590910,0.000000,0.576190,0.000000,0.499090,0.000000,0.531760,0.000000,0.553890,-0.015050,0.708240,-0.017690,0.496640,0.000000,0.442770,0.000000,0.409560,0.000000
75%,1.000000,0.0,1.000000,0.194185,1.000000,0.334655,0.969240,0.445675,0.953240,0.534195,0.957895,0.482375,0.955505,0.374860,0.919330,0.308975,0.935705,0.195285,0.899265,0.134370,0.894865,0.188760,0.911235,0.164630,0.905240,0.156765,0.999945,0.153535,0.883465,0.154075,0.857620,0.200120,0.813765,0.171660
max,1.000000,0.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [ ]:
ion.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 351 entries, 1 to 351
Data columns (total 35 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   V1      351 non-null    int64  
 1   V2      351 non-null    int64  
 2   V3      351 non-null    float64
 3   V4      351 non-null    float64
 4   V5      351 non-null    float64
 5   V6      351 non-null    float64
 6   V7      351 non-null    float64
 7   V8      351 non-null    float64
 8   V9      351 non-null    float64
 9   V10     351 non-null    float64
 10  V11     351 non-null    float64
 11  V12     351 non-null    float64
 12  V13     351 non-null    float64
 13  V14     351 non-null    float64
 14  V15     351 non-null    float64
 15  V16     351 non-null    float64
 16  V17     351 non-null    float64
 17  V18     351 non-null    float64
 18  V19     351 non-null    float64
 19  V20     351 non-null    float64
 20  V21     351 non-null    float64
 21  V22     351 non-null    float64
 22  V2

Quebrando a tabela em 70% para treinamento e 30% para validacao

In [ ]:
df = ion.copy()
df['Class'] = df['Class'].map({'good': 0, 'bad': 1})

df_train = df.sample(frac=0.7, random_state=0)
df_valid = df.drop(df_train.index)

Limpando os dados

In [ ]:
max_ = df_train.max(axis=0)
min_ = df_train.min(axis=0)

df_train = (df_train - min_) / (max_ - min_)
df_valid = (df_valid - min_) / (max_ - min_)
df_train.dropna(axis=1, inplace=True)
df_valid.dropna(axis=1, inplace=True)

In [ ]:
X_train = df_train.drop('Class', axis=1)
x_val = df_valid.drop('Class', axis=1)
y_train = df_train['Class']
y_val = df_valid['Class']

In [ ]:
X_train.values.tolist()
X_train.values

array([[1.      , 0.98794 , 0.44699 , ..., 0.09341 , 0.43084 , 0.095125],
       [1.      , 1.      , 1.      , ..., 0.32657 , 1.      , 0.674225],
       [1.      , 0.956765, 0.90793 , ..., 0.5     , 0.5     , 0.5     ],
       ...,
       [1.      , 1.      , 0.5369  , ..., 0.66246 , 1.      , 0.73356 ],
       [1.      , 0.86207 , 0.49458 , ..., 0.51478 , 0.895075, 0.50394 ],
       [1.      , 0.92067 , 0.40819 , ..., 0.37147 , 0.966615, 0.492875]])

In [ ]:
from torch.utils.data import TensorDataset, DataLoader
import torch

train_data = TensorDataset(torch.from_numpy(X_train.values), torch.from_numpy(y_train.values))
valid_data = TensorDataset(torch.from_numpy(x_val.values), torch.from_numpy(y_val.values))
train_loader = DataLoader(train_data, shuffle=True, batch_size=32, drop_last=True)
valid_loader = DataLoader(valid_data, shuffle=True, batch_size=32, drop_last=True)

In [ ]:
len(train_data[0]), len(train_data[1])

(2, 2)

Criando modelo com Pytorch


In [ ]:
!pip install -q -U keras-tuner

     |████████████████████████████████| 97 kB 3.6 MB/s 


In [ ]:
import kerastuner as kt

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  """Entry point for launching an IPython kernel.


In [ ]:
def model_builder(hp):
 
  model = keras.Sequential()
 
  # Tune the number of units in the first Dense layer
  # Choose an optimal value between 8-128
  hp_units_1 = hp.Int('hp_units_1', min_value=32, max_value=512, step=32)
  model.add(keras.layers.Dense(units=hp_units_1, activation='relu'))
  
  # Tune the number of hidden layers 2-10 and units per layer
  for i in range(hp.Int('num_layers', 2, 6)):     
    model.add(keras.layers.Dense(units=hp.Int('hp_units_' + str(i),
                                            min_value=16,
                                            max_value=256,
                                            step=4),
                               activation='relu'))

  model.add(keras.layers.Dense(16, activation='relu'))
  model.add(keras.layers.Dense(1, activation='sigmoid'))

  model.compile(optimizer=tf.optimizers.Adam(), 
                loss='binary_crossentropy', 
                metrics=['accuracy'])
  return model

In [ ]:
tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=50,
                     factor=3,
                     directory='mydir',
                     project_name='nils1_dl_course')

In [ ]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

In [ ]:

tuner.search(X_train, y_train,
             validation_data=(X_valid, y_valid),
             batch_size=512,
             epochs=40,
             verbose=1,
             callbacks=[stop_early])



Trial 128 Complete [00h 00m 04s]
val_accuracy: 0.9523809552192688

Best val_accuracy So Far: 1.0
Total elapsed time: 00h 15m 14s
INFO:tensorflow:Oracle triggered exit


INFO:tensorflow:Oracle triggered exit


In [ ]:
# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

In [ ]:

print('Best layers units: \n')
for i in range(2,6):
  print('Layer: ', i-1, ' , ', best_hps.get('hp_units_' + str(i)), ' units')

Best layers units: 

Layer:  1  ,  36  units
Layer:  2  ,  200  units
Layer:  3  ,  208  units
Layer:  4  ,  140  units


In [ ]:
# Build the model with the optimal hyperparameters and train it on the data for 50 epochs
model = tuner.hypermodel.build(best_hps)

history = model.fit(X_train, y_train, 
                    batch_size=512,
                    epochs=40, 
                    validation_split=0.2,
                    verbose=1)

val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))


Epoch 1/40
1/1 [==============================] - 1s 1s/step - loss: 0.6932 - accuracy: 0.5612 - val_loss: 0.6835 - val_accuracy: 0.6400
Epoch 2/40
1/1 [==============================] - 0s 34ms/step - loss: 0.6866 - accuracy: 0.5816 - val_loss: 0.6676 - val_accuracy: 0.6400
Epoch 3/40
1/1 [==============================] - 0s 34ms/step - loss: 0.6767 - accuracy: 0.5816 - val_loss: 0.6430 - val_accuracy: 0.6400
Epoch 4/40
1/1 [==============================] - 0s 39ms/step - loss: 0.6622 - accuracy: 0.5816 - val_loss: 0.6193 - val_accuracy: 0.6400
Epoch 5/40
1/1 [==============================] - 0s 41ms/step - loss: 0.6499 - accuracy: 0.5816 - val_loss: 0.5979 - val_accuracy: 0.6400
Epoch 6/40
1/1 [==============================] - 0s 44ms/step - loss: 0.6396 - accuracy: 0.5816 - val_loss: 0.5812 - val_accuracy: 0.6400
Epoch 7/40
1/1 [==============================] - 0s 40ms/step - loss: 0.6249 - accuracy: 0.5816 - val_loss: 0.5630 - val_accuracy: 0.6400
Epoch 8/40
1/1 [=============

In [ ]:
hypermodel = tuner.hypermodel.build(best_hps)

# Retrain the model
hypermodel.fit(X_train, y_train, 
               batch_size=64,
               epochs=best_epoch,
               validation_split=0.2)

Epoch 1/16
4/4 [==============================] - 1s 85ms/step - loss: 0.6854 - accuracy: 0.5816 - val_loss: 0.6270 - val_accuracy: 0.6400
Epoch 2/16
4/4 [==============================] - 0s 13ms/step - loss: 0.6497 - accuracy: 0.5816 - val_loss: 0.5846 - val_accuracy: 0.6400
Epoch 3/16
4/4 [==============================] - 0s 14ms/step - loss: 0.6159 - accuracy: 0.5816 - val_loss: 0.5201 - val_accuracy: 0.6400
Epoch 4/16
4/4 [==============================] - 0s 13ms/step - loss: 0.5541 - accuracy: 0.6327 - val_loss: 0.5062 - val_accuracy: 0.9400
Epoch 5/16
4/4 [==============================] - 0s 12ms/step - loss: 0.5009 - accuracy: 0.8367 - val_loss: 0.3766 - val_accuracy: 0.9400
Epoch 6/16
4/4 [==============================] - 0s 12ms/step - loss: 0.5070 - accuracy: 0.8214 - val_loss: 0.3596 - val_accuracy: 0.9400
Epoch 7/16
4/4 [==============================] - 0s 14ms/step - loss: 0.4223 - accuracy: 0.8469 - val_loss: 0.2906 - val_accuracy: 0.9000
Epoch 8/16
4/4 [===========

In [ ]:
eval_result = hypermodel.evaluate(X_train, y_train)
print("[test loss, test accuracy]:", eval_result)

8/8 [==============================] - 0s 2ms/step - loss: 0.2064 - accuracy: 0.9309
[test loss, test accuracy]: [0.20638971030712128, 0.9308943152427673]
